In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, END, START
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
from langchain_core.tools import tool
from dotenv import load_dotenv
import os
import json
from typing import Literal

load_dotenv()

In [ ]:
@tool
def pet_info(has_pet: bool | None, animal: str | None, pet_name: str | None, message: str | None) -> dict:
    """Capture whether the user has a pet, what kind of animal, and its name."""
    return {
        "has_pet": has_pet,
        "animal": animal,
        "pet_name": pet_name,
        "message": message or ""
    }

openai_api_key = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=openai_api_key)
llm_with_tool = llm.bind_tools([pet_info])

class State(TypedDict):
    has_pet: bool | None
    animal: str | None
    pet_name: str | None
    messages: Annotated[list[BaseMessage], add_messages]
    

def node_ask_about_pet(state: State):
    system_prompt = {
        "role": "system",
        "content": """You are a friendly assistant trying to find out if the user has a pet.
    Ask friendly, natural questions to figure out if they have a pet, what kind of animal it is, and the pet's name.
    
    At each step, call the pet_info tool with the information you currently have — even if it's incomplete.
    The tool has 4 fields: 
    - has_pet: true/false/null. True - has pet, False - has no pet, null uncertain if the user has a pet or not.
    - animal: e.g. dog, cat, etc. or null if unknown
    - pet_name: the name or null if unknown 
    - message: Use this field to add a reply message for the user with questions for missing data.
    
    
    - Call the tool on EVERY turn. Absolutely always call the tool on every turn, no exceptions, even if the user 
    is off-topic.
    - Ask questions to clarify missing information. 
    - If the user answers with irrelvant information call the tool and send a message that guides them back to the 
    conversation about the pet.
    - If the user says he has no pets set has_pet to False, and leave everything else as null. 
    - Once you know what kind of animal the pet is and it's name, set those values, but leave the message null.
    - If there is missing information ask a friendly question to find out the missing information.
    
    """
    }
    
    messages = [system_prompt] + state.get("messages", [])
    response = llm_with_tool.invoke(messages)

    try:
        tool_calls = response.additional_kwargs.get("tool_calls", [])
        if tool_calls:
            args = tool_calls[0].get("function", {}).get("arguments")
            if isinstance(args, str):
                args = json.loads(args)
            msg = args.get("message")
            return {
                "has_pet": args.get("has_pet"),
                "animal": args.get("animal"),
                "pet_name": args.get("pet_name"),
                "messages": add_messages(state["messages"], [AIMessage(content=msg)] if msg else [])
            }
        else:
            msg = response.content
            return state | {
                "messages": add_messages(state["messages"], [AIMessage(content=msg)] if msg else [])
            }
    except Exception as e:
        return state | {"messages": [AIMessage(content="Oops, something went wrong. Can you try rephrasing that?")]}


def node_get_user_input(state: State):
    user_input = input("User: ")
    if user_input.lower() in ["quit", "exit", "q", "bye", ""]:
        return state | {"messages": [HumanMessage(content="Goodbye!")]}  
    return {
        "messages": add_messages(state["messages"], [HumanMessage(content=user_input)])
    }
    

def node_final_message(state: State): 
    system_prompt = {
        "role": "system",
        "content": f"""You are a friendly assistant giving a final response to a user at the end of a chat about his or her pet. 
        Here's what you know about the user's pet status.
        
        The user has a pet: {state["has_pet"]}
        The type of animal: {state["animal"]}
        The name of the pet: {state["pet_name"]}
        
        Say something nice and then clearly say goodbye so that the user knows for sure that the conversation is over. 
    """
    }
    messages = [system_prompt]
    response = llm.invoke(messages)
    return state | {"messages": add_messages(state["messages"], [AIMessage(content=response.content)])}
    

def is_finished(state: State)  -> Literal["node_final_message", "node_get_user_input"]:
    match state:
        case {"has_pet": False}:
            return "node_final_message"
        case {"has_pet": True, "animal": str(), "pet_name": str()}:
            return "node_final_message"
        case _:
            return "node_get_user_input"


def build_graph():
    graph_builder = StateGraph(State)
    graph_builder.add_node("node_ask_about_pet", node_ask_about_pet)
    graph_builder.add_node("node_get_user_input", node_get_user_input)
    graph_builder.add_node("node_final_message", node_final_message)
    
    graph_builder.set_entry_point("node_ask_about_pet")
    graph_builder.add_edge("node_get_user_input", "node_ask_about_pet")
    graph_builder.add_conditional_edges("node_ask_about_pet", is_finished)
    graph_builder.add_edge("node_final_message", END)
    memory = MemorySaver()
    graph = graph_builder.compile(checkpointer=memory)
    return graph


def run_chatbot():
    config = {"configurable": {"thread_id": "1"}}
    state = {"has_pet": None, "animal": None, "pet_name": None, "messages": []}
    graph = build_graph()
    for event in graph.stream(state, config):
        for value in event.values():
            if "messages" in value and value["messages"]:
                last_msg = value["messages"][-1]
                if isinstance(last_msg, AIMessage):
                    print("Bot:", last_msg.content)
            state = value
    return {
        "has_pet": state["has_pet"],
        "animal": state["animal"],
        "pet_name": state["pet_name"],
    }

if __name__ == "__main__":
    pet_info = run_chatbot()
    print("\n\nPet Info: ", pet_info)